<a href="https://colab.research.google.com/github/HromovAndrey/II_Homework_mood_6_part_2/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_text_classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Завантажте усі датафрейми
* Добавте до кожного стовпчик `labels` з відповідним йому класом(bussiness, sports, education, etc). назви класів беріть з назв файлів
* Об'єднайте всі датафрейми в один, див [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
* Створіть стовпчик `text` в якому зберігатиметься текст для подальшої роботи. Можети вибрати такі варіанти:
  * Взяти один зі стовпчиків `headline`, `description`, `content`
  * Об'єднати тексти зі стовпчиків `headline`, `description`, `content`
* Видаліть стовпчики `headline`, `description`, `content`, `url`
* Розділіть дані на тренувальні та тестові
* Застосуйте `TfidfVectorizer`
* Натренуйте модель
* Виведіть результати

**Додатково**
* У `MyTokenizer` добавте обробку тексту за допомогою `nltk`, застусуйте один з варіантів:
  * стематизація
  * лематизація(використайте `nltk.pos_tag` для визначення частини мови)
  * обидва варіанти
* Використайте `n-gram` для класифікації(можливо доведеться збільшити `max_features`)
* Натренуйте різні моделі:
  * [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
  * [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)
  * [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) з параметром `kernel="linear"`
  * [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) з параметром `kernel="rbf"`
  * [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)
* Використайте `optuna` для підбору гіперпараметрів


# Дані про статті

In [13]:
import pandas as pd

df1 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/business_data.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/education_data.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/entertainment_data.csv")
df4 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/sports_data.csv")
df5 = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module6/data/technology_data.csv")

In [14]:
df1['labels'] = 'business'
df2['labels'] = 'education'
df3['labels'] = 'entertainment'
df4['labels'] = 'sports'
df5['labels'] = 'technology'

df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [15]:
df['text'] = df['headlines'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')

df.drop(['headlines', 'description', 'content', 'url'], axis=1, inplace=True)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], test_size=0.2, random_state=42)


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_tfidf, y_train)
y_pred_dt = dt_model.predict(X_test_tfidf)
print("DecisionTreeClassifier Report:")
print(classification_report(y_test, y_pred_dt))

nb_model = GaussianNB()
nb_model.fit(X_train_tfidf.toarray(), y_train)
y_pred_nb = nb_model.predict(X_test_tfidf.toarray())
print("GaussianNB Report:")
print(classification_report(y_test, y_pred_nb))

svc_linear_model = SVC(kernel="linear", random_state=42)
svc_linear_model.fit(X_train_tfidf, y_train)
y_pred_svc_linear = svc_linear_model.predict(X_test_tfidf)
print("SVC with linear kernel Report:")
print(classification_report(y_test, y_pred_svc_linear))

svc_rbf_model = SVC(kernel="rbf", random_state=42)
svc_rbf_model.fit(X_train_tfidf, y_train)
y_pred_svc_rbf = svc_rbf_model.predict(X_test_tfidf)
print("SVC with rbf kernel Report:")
print(classification_report(y_test, y_pred_svc_rbf))

mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train_tfidf, y_train)
y_pred_mlp = mlp_model.predict(X_test_tfidf)
print("MLPClassifier Report:")
print(classification_report(y_test, y_pred_mlp))


DecisionTreeClassifier Report:
               precision    recall  f1-score   support

     business       0.86      0.87      0.87       410
    education       0.91      0.90      0.91       393
entertainment       0.94      0.94      0.94       395
       sports       0.90      0.90      0.90       415
   technology       0.83      0.83      0.83       387

     accuracy                           0.89      2000
    macro avg       0.89      0.89      0.89      2000
 weighted avg       0.89      0.89      0.89      2000

GaussianNB Report:
               precision    recall  f1-score   support

     business       0.94      0.96      0.95       410
    education       0.98      0.96      0.97       393
entertainment       0.97      0.97      0.97       395
       sports       0.97      0.98      0.97       415
   technology       0.94      0.93      0.93       387

     accuracy                           0.96      2000
    macro avg       0.96      0.96      0.96      2000
 weighted 

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

# Загрузка необходимых ресурсов NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')  # Загружаем стоп-слова

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

class MyTokenizer:
    def __init__(self, use_lemmatization=True, use_stemming=False):
        self.use_lemmatization = use_lemmatization
        self.use_stemming = use_stemming

    def tokenize(self, text):
        words = nltk.word_tokenize(text)
        if self.use_lemmatization:
            words = [lemmatizer.lemmatize(word) for word in words]
        if self.use_stemming:
            words = [stemmer.stem(word) for word in words]
        words = [word for word in words if word.isalnum() and word not in stop_words]
        return ' '.join(words)

# Применение токенизатора к тексту
tokenizer = MyTokenizer(use_lemmatization=True, use_stemming=True)
df['text'] = df['text'].apply(tokenizer.tokenize)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    svc_c = trial.suggest_loguniform('svc_c', 1e-3, 1e3)
    svc_gamma = trial.suggest_loguniform('svc_gamma', 1e-4, 1e-1)

    model = SVC(C=svc_c, gamma=svc_gamma, kernel="rbf", random_state=42)
    score = cross_val_score(model, X_train_tfidf, y_train, cv=3, scoring='accuracy').mean()

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
print("Best trial parameters:", study.best_params)


ModuleNotFoundError: No module named 'optuna'